# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 6 2022 9:09AM,245694,10,0085959601,ISDIN Corporation,Released
1,Sep 6 2022 9:09AM,245694,10,0085959602,ISDIN Corporation,Released
2,Sep 6 2022 9:09AM,245694,10,0085959605,ISDIN Corporation,Released
3,Sep 6 2022 9:09AM,245694,10,0085959603,ISDIN Corporation,Released
4,Sep 6 2022 9:09AM,245694,10,0085959604,ISDIN Corporation,Released
5,Sep 6 2022 9:09AM,245694,10,0085959606,ISDIN Corporation,Released
6,Sep 6 2022 9:09AM,245694,10,0085959607,ISDIN Corporation,Released
7,Sep 6 2022 9:09AM,245694,10,0085959610,ISDIN Corporation,Released
8,Sep 6 2022 9:09AM,245694,10,0085959611,ISDIN Corporation,Released
9,Sep 6 2022 9:09AM,245694,10,0085959608,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
4,245689,Released,8
5,245690,Executing,1
6,245690,Released,5
7,245693,Released,64
8,245694,Released,48


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
245684,4.0,10.0
245689,NaN,8.0
245690,1.0,5.0
245693,NaN,64.0
245694,NaN,48.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
245682,0.0,4.0
245683,11.0,0.0
245684,4.0,10.0
245689,0.0,8.0
245690,1.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
245682,0,4
245683,11,0
245684,4,10
245689,0,8
245690,1,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,245682,0,4
1,245683,11,0
2,245684,4,10
3,245689,0,8
4,245690,1,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,245682,,4
1,245683,11,
2,245684,4,10
3,245689,,8
4,245690,1,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 6 2022 9:09AM,245694,10,ISDIN Corporation
48,Sep 6 2022 9:09AM,245693,10,ISDIN Corporation
112,Sep 6 2022 8:47AM,245690,10,Bio-PRF
118,Sep 6 2022 8:45AM,245689,20,Alumier Labs Inc.
126,Sep 6 2022 8:20AM,245684,10,ISDIN Corporation
140,Sep 6 2022 7:58AM,245682,20,"Exact-Rx, Inc."
144,Sep 6 2022 7:48AM,245683,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 6 2022 9:09AM,245694,10,ISDIN Corporation,,48
1,Sep 6 2022 9:09AM,245693,10,ISDIN Corporation,,64
2,Sep 6 2022 8:47AM,245690,10,Bio-PRF,1,5
3,Sep 6 2022 8:45AM,245689,20,Alumier Labs Inc.,,8
4,Sep 6 2022 8:20AM,245684,10,ISDIN Corporation,4,10
5,Sep 6 2022 7:58AM,245682,20,"Exact-Rx, Inc.",,4
6,Sep 6 2022 7:48AM,245683,19,"AdvaGen Pharma, Ltd",11,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 9:09AM,245694,10,ISDIN Corporation,48,
1,Sep 6 2022 9:09AM,245693,10,ISDIN Corporation,64,
2,Sep 6 2022 8:47AM,245690,10,Bio-PRF,5,1
3,Sep 6 2022 8:45AM,245689,20,Alumier Labs Inc.,8,
4,Sep 6 2022 8:20AM,245684,10,ISDIN Corporation,10,4
5,Sep 6 2022 7:58AM,245682,20,"Exact-Rx, Inc.",4,
6,Sep 6 2022 7:48AM,245683,19,"AdvaGen Pharma, Ltd",,11


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 9:09AM,245694,10,ISDIN Corporation,48,
1,Sep 6 2022 9:09AM,245693,10,ISDIN Corporation,64,
2,Sep 6 2022 8:47AM,245690,10,Bio-PRF,5,1
3,Sep 6 2022 8:45AM,245689,20,Alumier Labs Inc.,8,
4,Sep 6 2022 8:20AM,245684,10,ISDIN Corporation,10,4


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 9:09AM,245694,10,ISDIN Corporation,48.0,NaN
1,Sep 6 2022 9:09AM,245693,10,ISDIN Corporation,64.0,NaN
2,Sep 6 2022 8:47AM,245690,10,Bio-PRF,5.0,1.0
3,Sep 6 2022 8:45AM,245689,20,Alumier Labs Inc.,8.0,NaN
4,Sep 6 2022 8:20AM,245684,10,ISDIN Corporation,10.0,4.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 9:09AM,245694,10,ISDIN Corporation,48.0,0.0
1,Sep 6 2022 9:09AM,245693,10,ISDIN Corporation,64.0,0.0
2,Sep 6 2022 8:47AM,245690,10,Bio-PRF,5.0,1.0
3,Sep 6 2022 8:45AM,245689,20,Alumier Labs Inc.,8.0,0.0
4,Sep 6 2022 8:20AM,245684,10,ISDIN Corporation,10.0,4.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,982761,127.0,5.0
19,245683,0.0,11.0
20,491371,12.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,982761,127.0,5.0
1,19,245683,0.0,11.0
2,20,491371,12.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,127.0,5.0
1,19,0.0,11.0
2,20,12.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,127.0
1,19,Released,0.0
2,20,Released,12.0
3,10,Executing,5.0
4,19,Executing,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20
Status,,,
Executing,5.0,11.0,0.0
Released,127.0,0.0,12.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20
0,Executing,5.0,11.0,0.0
1,Released,127.0,0.0,12.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20
0,Executing,5.0,11.0,0.0
1,Released,127.0,0.0,12.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()